In [1]:
import keras

Using TensorFlow backend.


In [1]:
from keras.applications import VGG16
#Load the VGG model
vgg_conv = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

Using TensorFlow backend.


In [2]:
# Freeze the layers except the last 4 layers
for layer in vgg_conv.layers[:-4]:
    layer.trainable = False
 
# Check the trainable status of the individual layers
for layer in vgg_conv.layers:
    print(layer, layer.trainable)

<keras.engine.input_layer.InputLayer object at 0x7f796bf47908> False
<keras.layers.convolutional.Conv2D object at 0x7f7923edd128> False
<keras.layers.convolutional.Conv2D object at 0x7f7923edd208> False
<keras.layers.pooling.MaxPooling2D object at 0x7f7923690400> False
<keras.layers.convolutional.Conv2D object at 0x7f7923690278> False
<keras.layers.convolutional.Conv2D object at 0x7f79236b5cf8> False
<keras.layers.pooling.MaxPooling2D object at 0x7f7923668198> False
<keras.layers.convolutional.Conv2D object at 0x7f7923668080> False
<keras.layers.convolutional.Conv2D object at 0x7f79236083c8> False
<keras.layers.convolutional.Conv2D object at 0x7f7923626128> False
<keras.layers.pooling.MaxPooling2D object at 0x7f79235e0198> False
<keras.layers.convolutional.Conv2D object at 0x7f79235e0080> False
<keras.layers.convolutional.Conv2D object at 0x7f79235fe3c8> False
<keras.layers.convolutional.Conv2D object at 0x7f792359c128> False
<keras.layers.pooling.MaxPooling2D object at 0x7f7923554198>

In [3]:
from keras import models
from keras import layers
from keras import optimizers
 
# Create the model
model = models.Sequential()
 
# Add the vgg convolutional base model
model.add(vgg_conv)
 
# Add new layers
model.add(layers.Flatten())
model.add(layers.Dense(1024, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(2, activation='softmax'))
 
# Show a summary of the model. Check the number of trainable parameters
model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 7, 7, 512)         14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 25088)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              25691136  
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 2050      
Total params: 40,407,874
Trainable params: 32,772,610
Non-trainable params: 7,635,264
_________________________________________________________________


In [5]:
from keras.preprocessing.image import ImageDataGenerator
image_size=224
#train_dir = 'img/catsvsdogs'
#validation_dir = 'img/catsvsdogs'

train_dir = '/home/jaley/Downloads/all/train_seperate'
validation_dir = '/home/jaley/Downloads/all/train_seperate'


train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=20,
      width_shift_range=0.2,
      height_shift_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')
 
validation_datagen = ImageDataGenerator(rescale=1./255)
 
# Change the batchsize according to your system RAM
train_batchsize = 100
val_batchsize = 10
 
train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(image_size, image_size),
        batch_size=train_batchsize,
        class_mode='categorical')
 
validation_generator = validation_datagen.flow_from_directory(
        validation_dir,
        target_size=(image_size, image_size),
        batch_size=val_batchsize,
        class_mode='categorical',
        shuffle=False)

Found 25000 images belonging to 2 classes.
Found 25000 images belonging to 2 classes.


In [ ]:
# Compile the model
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['acc'])
# Train the model
history = model.fit_generator(
      train_generator,
      steps_per_epoch=train_generator.samples/train_generator.batch_size ,
      epochs=30,
      validation_data=validation_generator,
      validation_steps=validation_generator.samples/validation_generator.batch_size,
      verbose=1)
 
# Save the model
model.save('exp/5.2.1-exp/small_last4.h5')

Epoch 1/30
250/250 [==============================] - 7927s 32s/step - loss: 0.3701 - acc: 0.8433 - val_loss: 0.0984 - val_acc: 0.9619
Epoch 2/30
250/250 [==============================] - 6641s 27s/step - loss: 0.1587 - acc: 0.9378 - val_loss: 0.0839 - val_acc: 0.9678
Epoch 3/30
250/250 [==============================] - 6534s 26s/step - loss: 0.1328 - acc: 0.9491 - val_loss: 0.0910 - val_acc: 0.9717
Epoch 4/30
249/250 [============================>.] - ETA: 15s - loss: 0.1163 - acc: 0.9553